In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
%%capture
try: from torch_snippets import *
except:
  %pip -q install -U diffusers transformers accelerate torch-snippets lovely_tensors torchinfo pysnooper

In [ ]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
from IPython.display import HTML
from base64 import b64encode

pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

...

vae/config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

model.fp16.safetensors:   0%|          | 0.00/681M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
pipe.enable_model_cpu_offload()

In [ ]:

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
# pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

...

text_encoder/config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
1

1

In [ ]:
from diffusers.models.unet_3d_blocks import (
    CrossAttnDownBlock3D, CrossAttnUpBlock3D,
    TemporalConvLayer, TransformerTemporalModel
)

In [ ]:
!ln -s /usr/local/lib/python3.10/dist-packages/diffusers/models/resnet.py

In [ ]:
from torchinfo import summary
summary(pipe.unet, depth=4)

Debugging code

In [ ]:
from diffusers.models.unet_3d_blocks import (
    CrossAttnDownBlock3D, CrossAttnUpBlock3D,
    TemporalConvLayer, TransformerTemporalModel
)
from torch_snippets.trainer.hooks import print_module_ios_for
import pysnooper

prompt = 'bear running on the ocean'
negative_prompt = 'low quality'
video_duration_seconds = 3
num_frames = video_duration_seconds * 10 + 1

SNOOP = False
if SNOOP:
  pipe.og_call = pipe.__call__
  pipe.__call__ = pysnooper.snoop()(pipe.__call__)


keep = {'TemporalConvLayer', 'TransformerTemporalModel'}
with print_module_ios_for(pipe.unet, keep):
  video_frames = pipe(prompt, negative_prompt="low quality", num_inference_steps=1, num_frames=num_frames).frames

if SNOOP:
  pipe.__call__ = pipe.og_call

video_path = export_to_video(video_frames)

Generation code

In [ ]:
import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML

prompt = 'bear running on the ocean'
negative_prompt = 'low quality'
video_duration_seconds = 3
num_frames = video_duration_seconds * 25 + 1
video_frames = pipe(prompt, negative_prompt="low quality", num_inference_steps=25, num_frames=num_frames).frames

def display_video(video):
    fig = plt.figure(figsize=(4.2,4.2))  #Display size specification
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    mov = []
    for i in range(len(video)):  #Append videos one by one to mov
        img = plt.imshow(video[i], animated=True)
        plt.axis('off')
        mov.append([img])

    #Animation creation
    anime = animation.ArtistAnimation(fig, mov, interval=100, repeat_delay=1000)

    plt.close()
    return anime
video_path = export_to_video(video_frames)
video = imageio.mimread(video_path)  #Loading video
HTML(display_video(video).to_html5_video())  #Inline video display in HTML5

  0%|          | 0/25 [00:00<?, ?it/s]